<a href="https://colab.research.google.com/github/myarnwas/prompt-crafting-mayar/blob/main/mayar_gpt_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Function Calling

## 1.1. Provider Agnostic Function Calling

### Note :
Any LLm could be used here. But for simplification, we will use OpenAI's LLMs. However, we will only use the simple text completion API.

---

First of all, if we want to use OpenAI python library, we need to install it. We can do this by running the following command in our terminal:

```bash
pip install openai
```

Or simply running the next cell in this notebook:
- `%pip install openai` : install openai for the active python kernel
- `-q` : quiet mode, suppresses output

### 1.1.a Iinitialization

In [ ]:
%pip install openai python-dotenv -q

In [ ]:
import os

from dotenv import load_dotenv
from getpass import getpass
from openai import OpenAI

load_dotenv()                                   # This loads the environment variables. Make sure to have a .env file with your OpenAI API key.

False

In [ ]:
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key found in environment variables.")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
else :
    print("OpenAI API key not found in environment variables. Please set it in a .env file.")
    OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=OPENAI_API_KEY)

OpenAI API key not found in environment variables. Please set it in a .env file.
Enter your OpenAI API key: ··········


#### utils

In [ ]:
# ---------------------------------------------------------------------------- #
#            Change these values, they are initially set fot o4-mini           #
# ---------------------------------------------------------------------------- #
# Add input tokens cost
PER_MILLION_INPUT_TOKENS = 1.1      # 1.1 USD per million input tokens
# Add output tokens cost
PER_MILLION_OUTPUT_TOKENS = 4.4      # 4.4 USD per million output tokens

def estimate_cost(response, input_token_cost=PER_MILLION_INPUT_TOKENS, output_token_cost=PER_MILLION_OUTPUT_TOKENS):
    total_cost = 0

    total_cost += (response.usage.input_tokens / 1_000_000) * input_token_cost
    total_cost += (response.usage.output_tokens / 1_000_000) * output_token_cost

    # print(f"Request cost: ${total_cost:.4f}")
    print(f"\033[1mRequest cost: \033[94m{total_cost:.4f} $\033[0m")

In [ ]:
import json
import re

def extract_info_from_response(resopnse):
    # answer is between ```json ... ```
    json_string = re.search(r'```json(.*?)```', resopnse.output[0].content[0].text, re.DOTALL).group(1)
    json_string = json_string.strip()   # Remove any trailing whitespace
    json_answer = json.loads(json_string)             # This will raise an error if the JSON is not valid. Maybe you should optimize the prompt or simply retry the request.
    return json_answer

### 1.1.b Function and Template definition

In [ ]:
def get_calendar(user_id):
    """
    This function gets the calendar of a user.
    :param user_id: The ID of the user.
    :return: The calendar of the user in a dictionary format.
    """
    # For simplification, we are using a simple static calendar.
    if user_id == 332:
        calendar = {
            "Today" : {
                "09:00-10:00" : "Meeting with Client",
                "10:00-16:00" : "Work on Project",
                "16:00-17:00" : "Call with Team"
            },
            "Tomorrow" : {
                "10:00-11:00" : "Free for meetings",
                "14:00-16:00" : "Work on Project",
                "16:00-17:00" : "Call with Team"
            },
        }
    else:
        calendar = {
            "Today" : {
                "09:00-10:00" : "Meeting with Client",
                "10:00-16:00" : "Work on Project",
                "16:00-17:00" : "Call with Team"
            },
            "Tomorrow" : {
                "09:00-16:00" : "Work on Project",
                "16:00-17:00" : "Call with Team"
            },
        }
    return calendar

def schedule_meeting(user_id, customer_id, start_time, end_time):
    """
    This function schedules a meeting for a user.
    :param user_id: The ID of the user.
    :param customer_id: The ID of the customer.
    :param start_time: The start time of the meeting.
    :param end_time: The end time of the meeting.
    :return: A confirmation message.
    """
    # Some code here that  schedules a meeting.
    ...
    # For simplification, we are just returning a confirmation message.
    print(f"Meeting scheduled for user {user_id} with customer {customer_id} from {start_time} to {end_time}.")
    return True

available_tools = {
    "get_calendar": get_calendar,
    "schedule_meeting": schedule_meeting
}

In [ ]:
# Tools list the functions that can be called by the assistant.
TOOLS = """
1. Function: `get_calendar`
   Description: Given a user ID, it returns the user's availability.
   Parameters:
   - `user_id` (int): The ID of the user whose calendar should be retrieved.

2. Function: `schedule_meeting`
   Description: Schedules a meeting between a user and a customer.
   Parameters:
   - `user_id` (int): The ID of the user to schedule the meeting with.
   - `customer_id` (int): The ID of the customer.
   - `start_time` (string, ISO 8601 format): The start datetime of the meeting.
   - `end_time` (string, ISO 8601 format): The end datetime of the meeting.
"""

# Instructions are here to force the model to normalize the output to a specific format.
INSTRUCTIONS = """
You must decide how to handle the input request. You have two options:

### Option 1: Call a Function
If the task requires using one of the provided functions, respond **only** with the following JSON format:

```json
{
  "function_call": {
    "name": "<function_name>",
    "arguments": {
      "<parameter_1>": <value_1>,
      "<parameter_2>": <value_2>,
      ...
    }
  }
}
```

### Option 2: Provide a Final Answer
If the request can be fully handled without calling a function, return a final answer in this format:

```json
{
  "final_answer": "<your response to the user in natural language>"
}
```
Do not mix both formats. Choose one based on what is appropriate for the task. Do not add explanations or extra content outside the chosen JSON format.
"""

# The input contains the user's request
INPUT = """
Schedule a meeting between "Ali" (customer_id = 223) and one of our managers sometime tomorrow:
- Mohammad : user_id = 239
- Soufiane : user_id = 332
- Amine : user_id = 321
"""

# Will contain the execution
INFO = """"""

# The generated prompt
prompt_template = """
You are an intelligent assistant that can use external tools (functions) to help perform actions. You are provided with the following functions:

FUNCTIONS:
{TOOLS}

INSTRUCTIONS:
{INSTRUCTIONS}

INFO:
{INFO}

INPUT:
{INPUT}
"""

### 1.1.c Start the function calling

In [ ]:
response = client.responses.create(
  model="gpt-4.1-2025-04-14",
  input=[{
      "role": "user",
      "content": prompt_template.format(TOOLS=TOOLS, INSTRUCTIONS=INSTRUCTIONS, INFO=INFO, INPUT=INPUT)
  }]
)
estimate_cost(response, input_token_cost=2, output_token_cost=8)


Request cost: 0.0012 $


In [ ]:
json_answer = extract_info_from_response(response)
json_answer

{'function_call': {'name': 'get_calendar', 'arguments': {'user_id': 239}}}

As you can see, now the LLM asked us to provide more information. Lets provide it with the information it asked for.

In [ ]:
tools_answer = available_tools[json_answer["function_call"]["name"]](
    **json_answer["function_call"]["arguments"]
)
INFO += f"""
{json_answer["function_call"]["name"]}({json_answer["function_call"]["arguments"]}) = {tools_answer}
"""
print(INFO)


get_calendar({'user_id': 239}) = {'Today': {'09:00-10:00': 'Meeting with Client', '10:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}, 'Tomorrow': {'09:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}}



In [ ]:
response = client.responses.create(
  model="gpt-4.1-2025-04-14",
  input=[{
      "role": "user",
      "content": prompt_template.format(TOOLS=TOOLS, INSTRUCTIONS=INSTRUCTIONS, INFO=INFO, INPUT=INPUT)
  }]
)
estimate_cost(response, input_token_cost=2, output_token_cost=8)

Request cost: 0.0013 $


In [ ]:
json_answer = extract_info_from_response(response)
json_answer

{'function_call': {'name': 'get_calendar', 'arguments': {'user_id': 332}}}

### 1.1.d Master the function calling with loops

We will continously prompt the LLM until we reach a final response.

In [ ]:
class Assitant:
    def __init__(self,
                 tools=TOOLS,
                 instructions=INSTRUCTIONS,
                 input=INPUT,
                 prompt_template=prompt_template,
                 available_tools=available_tools,
                 model="gpt-4.1-2025-04-14",
                 model_input_token_cost=2,
                 model_output_token_cost=8,
                 max_steps=15,
                 client=client,
                 verbose=False,
        ):
        self.tools = tools
        self.instructions = instructions
        self.input = input
        self.prompt_template = prompt_template
        self.available_tools = available_tools
        self.model = model
        self.model_input_token_cost = model_input_token_cost
        self.model_output_token_cost = model_output_token_cost
        self.max_steps = max_steps
        self.client = client
        self.verbose = verbose
        self.info = ""
        self.total_cost = 0

    def _estimate_cost(self, response):
        """
        This function estimates the cost of a request.
        :param response: The response from the model.
        :return: The total cost of the request.
        """
        self.total_cost += (response.usage.input_tokens / 1_000_000) * self.model_input_token_cost
        self.total_cost += (response.usage.output_tokens / 1_000_000) * self.model_output_token_cost
        return self.total_cost

    def _step(self):
        response = self.client.responses.create(
          model=self.model,
          input=[{
              "role": "user",
              "content": self.prompt_template.format(TOOLS=self.tools, INSTRUCTIONS=self.instructions, INFO=self.info, INPUT=self.input)
          }]
        )
        self._estimate_cost(response)
        json_answer = extract_info_from_response(response)
        tools_answer = self.available_tools[json_answer["function_call"]["name"]](
            **json_answer["function_call"]["arguments"]
        )
        self.info += f"""
{json_answer["function_call"]["name"]}({json_answer["function_call"]["arguments"]}) = {tools_answer}
"""
        if self.verbose:
            print(self.info)
        return json_answer

    def stop_condition(self, json_answer):
        """
        The stop condition for the assistant.
        When the the function "schedule_meeting" is called or when the final answer is provided.
        """
        if "final_answer" in json_answer:       # The final answer, could be a message of impossibility or a confirmation.
            return True
        if json_answer["function_call"]["name"] == "schedule_meeting":  # When the meeting is scheduled
            return True
        return False

    def run(self):
        """
        This function runs the assistant until it reaches a stop condition or the maximum number of steps.
        :return: The final answer from the assistant.
        """
        for step in range(self.max_steps):
            json_answer = self._step()
            if self.stop_condition(json_answer):
                break
        return self.info, self.total_cost, step

Let's put it in action for now. It should be indicating that the meeting have been scheduled with the user 332

In [ ]:
assitant = Assitant(verbose=False)
info, total_cost, steps = assitant.run()
print(f"Total cost: {total_cost:.4f} $")
print(f"Total steps: {steps}")
print("--"*100)
print("INFO :")
print(info)

Meeting scheduled for user 332 with customer 223 from 2024-06-14T10:00:00 to 2024-06-14T11:00:00.
Total cost: 0.0061 $
Total steps: 3
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
INFO :

get_calendar({'user_id': 239}) = {'Today': {'09:00-10:00': 'Meeting with Client', '10:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}, 'Tomorrow': {'09:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}}

get_calendar({'user_id': 332}) = {'Today': {'09:00-10:00': 'Meeting with Client', '10:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}, 'Tomorrow': {'10:00-11:00': 'Free for meetings', '14:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}}

get_calendar({'user_id': 321}) = {'Today': {'09:00-10:00': 'Meeting with Client', '10:00-16:00': 'Work on Project', '16:00-17:00': 'Call with Team'}, 'Tomorr

## 1.2. Provider Specific Function Calling

### 1.2.1. OpenAI Function Calling

First of all, if we want to use OpenAI python library, we need to install it. We can do this by running the following command in our terminal:

```bash
pip install openai
```

Or simply running the next cell in this notebook:
- `%pip install openai` : install openai for the active python kernel
- `-q` : quiet mode, suppresses output

In [ ]:
%pip install openai python-dotenv -q

First lets initialize our libraries

In [ ]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()                                   # This loads the environment variables. Make sure to have a .env file with your OpenAI API key.

False

In [ ]:
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key found in environment variables.")
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
else :
    print("OpenAI API key not found in environment variables. Please set it in a .env file.")
    OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=OPENAI_API_KEY)

OpenAI API key not found in environment variables. Please set it in a .env file.
Enter your OpenAI API key: ··········


In [ ]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

response = client.responses.create(
    model="gpt-4.1-2025-04-14",
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Paris, France"}', call_id='call_6iok1nP09tmGTDFa8R56EZoz', name='get_weather', type='function_call', id='fc_686a087a8cdc81a28314f969f1780bc70628cd2afe26ab18', status='completed')]


#### Estimation of the cost. Reasoning and Caching is not taken into account.

In [ ]:
total_cost = 0

# Add input tokens cost
PER_MILLION_INPUT_TOKENS = 1.1      # 1.1 USD per million input tokens
total_cost += (response.usage.input_tokens / 1_000_000) * PER_MILLION_INPUT_TOKENS
# Add output tokens cost
PER_MILLION_OUTPUT_TOKENS = 4.4      # 4.4 USD per million output tokens
total_cost += (response.usage.output_tokens / 1_000_000) * PER_MILLION_OUTPUT_TOKENS

# print(f"Request cost: ${total_cost:.4f}")
print(f"\033[1mRequest cost: \033[94m{total_cost:.4f} $\033[0m")

Request cost: 0.0001 $


In [ ]:
response.usage

ResponseUsage(input_tokens=59, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=17, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=76)

Now as you can see, we can simply repeat the same logic as before, but this time we will use the OpenAI function calling API. It is always helpful to create a specific class for the function calling or for each use case. FOr OpenAI, we recommend to follow the [OpenAI documentation for Function Calling](https://platform.openai.com/docs/guides/function-calling).

The interest of using the Provider Specific Function Calling is that it is more efficient (cheaper) and it is more reliable. It is also more flexible as it allows you to define the function signature and the parameters that the LLM can use.

# TODO :
Now you have learned the basics of function calling and how to use OpenAI API. Now create a class like the ``Assitant`` class above to create a weather assistant app. You can create multiple functions like ``get_current_weather``, ``get_forecast``, ``get_air_quality``, etc. and use the OpenAI function calling API to call them. You can also use the OpenAI function calling API to call external APIs like OpenWeatherMap or WeatherAPI. The user should be able to ask for whether to go eat outside or not, to swim or not, etc.

# 1.3. Conclusion

Function Calling is independent of the LLM provider, it is even possible to use multiple providers mixed with some private or on-premise LLMs. However, it is more efficient and reliable to use the provider's specific function calling API.